In [174]:
import os
import warnings
import rasterio
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

In [176]:
# Чтение доп данных о пожарах от МЧС России
thermopoints = pd.read_csv('thermopoints.csv', delimiter=';')

,dt,type_name,type_id,lon,lat
0,2012-03-13,Природный пожар,4,131.5866,47.8662
1,2012-03-13,Природный пожар,4,131.5885,47.8809
2,2012-03-13,Лесной пожар,3,131.9871,48.4973
3,2012-03-13,Природный пожар,4,131.9031,43.6277
4,2012-03-13,Природный пожар,4,131.5706,47.8581
...,...,...,...,...,...
660249,2021-09-10,Лесной пожар,3,118.5451,64.7475
660250,2021-09-10,Лесной пожар,3,118.3046,64.7629
660251,2021-09-10,Лесной пожар,3,117.9681,65.7394
660252,2021-09-10,Лесной пожар,3,119.0462,64.7541


In [177]:
def print_geotiff_info(file_path):
    try:
        with rasterio.open(file_path) as src:
            # Основные метаданные
            print(f"File Path: {file_path}")
            print(f"Driver: {src.driver}")
            print(f"Width: {src.width}")
            print(f"Height: {src.height}")
            print(f"Count (Bands): {src.count}")
            print(f"CRS: {src.crs}")
            print(f"Transform: {src.transform}")
            print(f"Bounding Box: {src.bounds}")
            print(f"Datum: {src.dtypes}")
            # Информация по каждому каналу
            for i in range(1, src.count + 1):
                band = src.read(i)
                print(f"\nBand {i}:")
                print(f"  Data Type: {src.dtypes[i - 1]}")
                print(f"  Min Value: {band.min()}")
                print(f"  Max Value: {band.max()}")
                print(f"  Mean Value: {band.mean()}")
                print(f"  Standard Deviation: {band.std()}")
    except Exception as e:
        print(f'Error: {e}')

In [178]:
def getBox(path_tiff):
    with rasterio.open(path_tiff) as src:
        bbox = src.bounds
        long = bbox.left
        lat = bbox.top
        bottom = -bbox.bottom
        right = bbox.right

        coeff = 1 / (2 * np.pi / 360 * 6378.137) / 1000

        new_lat = lat + bottom * coeff
        new_long = long + right * coeff / np.cos(lat * (np.pi / 180))

        res = [lat, new_lat, long, float(new_long)]

    return res

### Создание датасета с вегетационными индексами

In [180]:
# Определяем столбцы
veg_index = ['NDVI', 'GNDVI', 'DVI', 'OSAVI', 'ExG', 'ExR', 'NDI', 'EVI']
columns = ['date', 'sector_id', 'lat', 'lon', 'B02', 'B03', 'B04', 'B08', 'fire'] + veg_index

In [181]:
def getDaysWithFire(thermopoints, path_tiff):
    '''Функция для получения дней с пожарами в секторе'''
    lat_long = getBox(path_tiff)
    
    return lat_long, thermopoints.loc[(thermopoints['lat'] > lat_long[0]) & (thermopoints['lat'] < lat_long[1]) & (thermopoints['lon'] > lat_long[2]) & (thermopoints['lon'] < lat_long[3]), 'dt'].values


def getBands(path_tiff, key_id):
    '''Функция для получения данных по каналам и вегетационным индексам из tiff файла'''
    
    res = pd.DataFrame(columns=columns)
    box = getBox(path_tiff)
    
    # Чтение данных из tiff файла
    with rasterio.open(path_tiff) as src:
        blue = src.read(1)
        green = src.read(2)
        red = src.read(3)
        nir = src.read(4)
        mask = src.read(5)

    rows = blue.shape[0]
    cols = blue.shape[1]

    
    res['B02'] = red.flatten()
    res['B03'] = green.flatten()
    res['B04'] = blue.flatten()
    res['B08'] = nir.flatten()
    res['fire'] = mask.flatten()
    res['date'] = path_tiff.split('/')[-1].split('.')[0]
    res['sector_id'] = key_id
    
    # Расчет вегетационных индексов по данным из tiff файла
    res['NDVI'] = (res['B08'] - res['B04']) / (res['B08'] + res['B04'])
    res['GNDVI'] = (res['B08'] - res['B03']) / (res['B08'] + res['B03'])
    res['DVI'] = res['B08'] - res['B04']
    res['OSAVI'] = 1.16 * (res['B08'] - res['B04']) / (res['B08'] + res['B04'] + 0.16)
    res['ExG'] = (2 * res['B03'] - res['B04'] - res['B02']) / (res['B02'] + res['B03'] + res['B04'])
    res['ExR'] = (1.4 * res['B04'] - res['B03']) / (res['B02'] + res['B03'] + res['B04'])
    res['NDI'] = (res['B03'] - res['B04']) / (res['B03'] + res['B04'])
    res['EVI'] = 2.5 * (res['B08'] - res['B04']) / (res['B08'] + 6 * res['B04'] - 7.5 * res['B02'] + 1)

    
    res['lat'] = np.repeat(np.linspace(box[0], box[1], num=rows), cols)
    res['lon'] = np.array([np.linspace(box[2], box[3], num=cols) for _ in range(rows)]).flatten()
    
    return res


def getMergedDataset():
    ''' Функция для получения объединенного датасета по всем секторам'''
    df = pd.DataFrame(columns=columns)

    for i in range(21):
        if i < 10:
            n = '0' + str(i)
        path_to_tiff = f'./train_dataset_train_correct/{n}/' + list(filter(lambda x : '.tiff' in x, os.listdir(f'./train_dataset_train_correct/{n}/')))[0]
        df = pd.concat([df, getBands(path_to_tiff, i)], axis=0)

    return df.reset_index(drop=True)


# Выполнение функции
res = getMergedDataset()

# Сохранение датасета
res.to_csv('dataset_veg_index.csv', index=True)

### Опредение леса по спутниковым снимкам.

In [ ]:
# Заданный словарь с пороговыми значениями NDVI для каждого месяца.
# NDVI: май - 0.81, июнь - 0.845, июль - 0.85, август - 0.86, сентярбь - 0,66.
NDVI_limits = {'05': 0.81, '06': 0.845, '07': 0.85, '08': 0.86, '09': 0.66}

# Функция для замены данных в переменной fil_index в соответствии с огранчениями NDVI_limits
def replace_data(fil_index, NDVI_limits, month):
    for i in range(fil_index.size):
            if fil_index[i] == None:
              print(1)
            if NDVI_limits[month[i]] <= fil_index[i] <= 1:
              fil_index[i] = 1
            elif 0 <= fil_index[i] < NDVI_limits[month[i]]:
              fil_index[i] = 0
            else:
              fil_index[i] = 2
    return fil_index


# Применяем функцию к данным
month = res['date'].apply(lambda x : x.split('-')[1]).values
fil_index = res['NDVI'].values
forest = replace_data(fil_index, NDVI_limits, month)

# Поле forest - данные о лесах, где значение 1 - леса
res.rename(columns={'NDVI': 'forest'}, inplace=True)

# Сохраняем результат в файл
# res.to_csv('dataset_veg_index.csv', index=True)

,date,sector_id,lat,lon,B02,B03,B04,B08,forest,fire
0,2021-06-06,0,56.299073,71.474895,34.0,25.0,19.0,53.0,0.0,0.0
1,2021-06-06,0,56.299073,71.475063,26.0,20.0,15.0,39.0,0.0,0.0
2,2021-06-06,0,56.299073,71.475231,31.0,23.0,17.0,47.0,0.0,0.0
3,2021-06-06,0,56.299073,71.475399,33.0,24.0,18.0,49.0,0.0,0.0
4,2021-06-06,0,56.299073,71.475567,25.0,19.0,15.0,38.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1022371,2021-06-16,20,54.836132,73.631562,17.0,16.0,11.0,49.0,0.0,0.0
1022372,2021-06-16,20,54.836132,73.631727,16.0,16.0,11.0,49.0,0.0,0.0
1022373,2021-06-16,20,54.836132,73.631893,17.0,16.0,11.0,50.0,0.0,0.0
1022374,2021-06-16,20,54.836132,73.632059,17.0,16.0,11.0,52.0,0.0,0.0
